In [ ]:
import pandas as pd
import numpy as np
from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True)
import matplotlib.pyplot as plt
import os, sys, subprocess
%load_ext watermark

In [ ]:
%watermark

In [ ]:
#parameters
workingdir=r"/Users/filippo/Developer/tesi/docker"
execdir = r"/Users/filippo/Developer/tesi/docker"
#dockerurl = "cloudadm@193.144.35.220:/home/cloudadm/drive"
dockerurl = "filippo.valle@c3s.unito.it:"

In [ ]:
os.chdir(workingdir)

# Get the data
[Table_creation](Table_creation.ipynb) does this for you

# Get annotations

In [ ]:
df_file=pd.read_csv("files.dat", index_col=[0])
df_file.head()

In [ ]:
df_file.groupby(by='primary_site').count()

In [ ]:
pc=pd.read_csv("%s/genes.txt"%execdir, index_col=[0])
pc=pc[pc['type_of_gene']=='protein-coding']
pc.head()

# Run statistics
```python
import tacos
tacos.statistics()
tacos.shuffleLabels()
```

CHECK **BioParameters.h** before compiling

In [ ]:
df=pd.read_csv("mainTable.csv", index_col=0,header=0)

In [ ]:
df.apply(lambda x: len(np.nonzero(x)[0])/len(x), axis=1).to_csv("O.dat", header=False, index=False)
df.apply(lambda x: np.sum(x), axis=1).to_csv("A.dat", header=False, index=False)
df.apply(lambda x: np.sum(x), axis=0).to_csv("vocabulary_size.dat", header=False, index=False)
pd.DataFrame(data=[df.apply(lambda x: np.sum(x), axis=0).values,df.apply(lambda x: len(np.nonzero(x)[0]), axis=0).values]).transpose().to_csv("heaps.dat",header=False,index=False)
pd.DataFrame(data=[df.apply(lambda x: np.average(x), axis=1).values,df.apply(lambda x: np.var(x), axis=1).values,df.apply(lambda x: np.nan, axis=1).values], columns=df.index, index=['mean','variance','type_of_gene']).transpose().to_csv("meanVariances.csv",header=True,index=True)

In [ ]:
os.chdir(workingdir)
mv = pd.read_csv("meanVariances.csv", index_col=0)
o=pd.read_csv("O.dat", header=None)
mv.insert(3,'occurrence',o.values)
mv.to_csv("meanVariances.csv")

# Analyse table and select genes

[Table_Analyser.ipynb](Table_Analyzer.ipynb)

In [ ]:
os.chdir(execdir)
from tableanalyser import scalinglawsandoverexpressed

In [ ]:
scalinglawsandoverexpressed(workingdir)

# Make bipartite network
```bash
python addo.py
```
```python
import tacos
tacos.makegraph()
```

In [ ]:
os.chdir(workingdir)
os.system("./data_mining 0")
os.system("./data_mining 2")
os.system("python addo.py")
os.system("./data_mining 7")

In [ ]:
os.system("scp graph.xml.gz %s/."%dockerurl)

[hSBM.ipynb]()

# Run stochastic block model
```
docker run -it -p8888:8888 -u user -v $(PWD)/drive/:/home/user/ fvalle01/hsbm bash
jupyter notebook --ip0.0.0.0 --allow-root
```

download the data

In [ ]:
os.chdir(workingdir)
os.system("mkdir -p topsbm")
os.system("scp %s/* topsbm/."%dockerurl)

# Run cluster analysis

In [ ]:
os.chdir(execdir)
from hsbmpy import clusteranalysis, get_scores, add_score_lines, getclustersizesarray
#geneontology

In [ ]:
os.chdir(execdir)
import hsbmpy, importlib
importlib.reload(hsbmpy)
from hsbmpy import clusteranalysis, get_scores, add_score_lines, getclustersizesarray
os.chdir(workingdir)

In [ ]:
labels = df_file.columns
clusteranalysis(workingdir, labels, l=5)

In [ ]:
#scores = get_scores(workingdir,labels, L=5, verbose=False)
fig=plt.figure(figsize=(12,8))
ax = fig.subplots(1)
add_score_lines(ax,scores,labels=[k for k in scores.keys()],xl=getclustersizesarray(workingdir,l=5))
plt.show()
fig.savefig("%s/metric.pdf"%(workingdir))

In [ ]:
geneontology.topicanalysis()

# Clean

In [ ]:
os.chdir(workingdir)
os.system("rm -rf data_mining addo.py")